In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv

from os import listdir
from os.path import isfile,join

import pickle


def load_image(img_name):
    
    img=cv.imread(img_name,cv.IMREAD_COLOR)

    return img


def fill_in(img):
    height,width=img.shape[0],img.shape[1]
    m=np.max((height,width))

    img_padded=255*np.ones((m,m,3))
    if width>height:
        img_padded[0:height,:,:]=img
    elif height>width:
        img_padded[:,0:width,:]=img
    else:
        return img

    return np.uint8(img_padded)


def resize(img_padded):

    c1=img_padded[:,:,0]
    c2=img_padded[:,:,1]
    c3=img_padded[:,:,2]

    c1=cv.resize(c1,(128,128),cv.INTER_CUBIC)
    c2=cv.resize(c2,(128,128),cv.INTER_CUBIC)
    c3=cv.resize(c3,(128,128),cv.INTER_CUBIC)

    resized=np.zeros((128,128,3),dtype=np.uint8)
    resized[:,:,0]=np.uint8(c1)
    resized[:,:,1]=np.uint8(c2)
    resized[:,:,2]=np.uint8(c3)

    return resized


def normalize(arr):
    arr = arr.astype('float')
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return np.uint8(arr)



def flat_and_label(img_folder_name,idx):
    img=load_image(img_folder_name)
    padded_img=fill_in(img)
    resized=resize(padded_img)
    normalized=normalize(resized)
    flattened=normalized.ravel()
    flattened=np.append(flattened,idx)

    return flattened


parent_dir='../../data/gear_images/'
folder=[f for f in listdir(parent_dir)]
counter=0 
for folder_name in folder:
    counter+= len([f for f in listdir(parent_dir+folder_name) if isfile(join(parent_dir+folder_name,f))])


images=np.empty((counter,128*128*3+1)) 
labels={}
counter2=0
for idx,folder_name in enumerate(folder):
    labels[folder_name]=idx
    print(idx)
    for f in listdir(parent_dir+folder_name):
        if isfile(join(parent_dir+folder_name,f)):
            img_path=parent_dir+folder_name+'/'+f
            images[counter2,:]=flat_and_label(img_path,idx)
            counter2+=1
           
with open('../../data/pre-processed-data/labels.pkl','wb') as f:
    pickle.dump(labels,f)
        
np.save('../../data/pre-processed-data/pre_processed_images',images)


FileNotFoundError: [Errno 2] No such file or directory: '../../data/gear_images/'